In [17]:

# coding: utf-8

### Loading the data

# First we set up our imports:

# In[ ]:
import os
from samson_const import *
import matplotlib
import yt
import numpy as np
import yt.units as units
import pylab
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib import rcParams
from yt.fields.particle_fields import add_volume_weighted_smoothed_field
from matplotlib.colors import LogNorm
from textwrap import wrap
from readsnap_cr import readsnapcr
import Sasha_functions as SF
from samson_functions import *
from yt.frontends.gizmo.api import GizmoDataset



#rcParams['figure.figsize'] = 15, 5
rcParams['font.size']=18
rcParams['font.family']='serif'
#rcParams.update({'figure.autolayout': True})
rcParams['axes.linewidth'] = 2
rcParams['pdf.fonttype'] = 42
rcParams['text.usetex']=True
rcParams['legend.fontsize']=10
#rcParams['ps.fonttype'] = 42
#rcParams['ps.useafm'] = True
#rcParams['pdf.use14corefonts'] = True
#rcParams['axes.unicode_minus']=False



def add_inner_title(ax, title, loc, size=None, **kwargs):
    from matplotlib.offsetbox import AnchoredText
    from matplotlib.patheffects import withStroke
    if size is None:
        size = dict(size=plt.rcParams['legend.fontsize'])
    at = AnchoredText(title, loc=loc, prop=size,
                      pad=0., borderpad=0.5,
                      frameon=False, **kwargs)
    ax.add_artist(at)
    at.txt._text.set_path_effects([withStroke(foreground="w", linewidth=3)])
    return at


# First we load the data set, specifying both the unit length/mass/velocity, as well as the size of the bounding box (which should encapsulate all the particles in the data set)
# 
# At the end, we flatten the data into "ad" in case we want access to the raw simulation data

# >This dataset is available for download at http://yt-project.org/data/GadgetDiskGalaxy.tar.gz (430 MB).

# In[ ]:



Nsnapstring='005'
#Nsnap=100
#Nsnap=500
#Nsnap=580


fns=[\
'/home/tkc004/oasis/bw/CRdifftest/dc28g3D_old/output/snapshot_'+Nsnapstring+'.hdf5',\
'/home/tkc004/oasis/bw/CRdifftest/M1dc28g3D_c1000_useflux_Jianhll_referee/output/snapshot_'+Nsnapstring+'.hdf5',\
#'/home/tkc004/oasis/bw/CRdifftest/M1dc28g3D_c1000_usefluxy2000_Jianhll/output/snapshot_'+Nsnapstring+'.hdf5',\
#'/home/tkc004/oasis/bw/CRdifftest/M1dc28g3D_c1000_usefluxy2000_Jianhll_referee/output/snapshot_'+Nsnapstring+'.hdf5',\
#'/home/tkc004/oasis/bw/CRdifftest/M1dc28g3D_c500_useflux_Jianhll/output/snapshot_014.hdf5',\
#'/home/tkc004/oasis/bw/CRdifftest/M1dc28g3D_c500_useflux_Jianhll_referee/output/snapshot_014.hdf5',\
#'/home/tkc004/oasis/CRdifftest/M1dc28g2D_c500_useflux_Jianhll_referee/output/snapshot_020.hdf5',\
    ]


#plotit=[]

plotit =[\
#'Original',
#'Referee'
'Zeroth moment',
'Two moment',
        ]

#plotit = [\
#'Hydro no CR',\
#'MHD no CR',\
#'Advection',\
#r'$\kappa$=3e27',\
#r'$\kappa$=3e28',\
#r'$\kappa$=3e29',\
#r'MHD             Streaming',\
##STRLL',\
#r'MHD $\kappa$=3e28',\
#'MHD stream'
#r'MHD $\kappa$=3e28 Streaming'\
#'Diffusion      Streaming'\
#]

#plotit = [\
#'STREAM',\
#'STREAMLLVA',\
#'STREAMLL4VA'\
#]

#plotit = ['DC=3e27','DC=3e28','DC=3e29']
#plotit = ['DC=0','DC=3e27','DC=3e28']


plotupt=['','']

#plotupt=['','SMC','']
#plotupt=['','MHD']
#plotupt=['Milky Way']
#plotupt=['Milky Way', '']
#plotupt=[\
#'',\
#'Starburst',\
#'Dwarf',\
#r'$L\star$ Galaxy',\
#'',\
#'',\
#'',\
#'',\
#'',\
#'',\
#''\
#]
#plotupt=['','Milky Way', '']
#plotupt=['','Dwarf','']


#quivercolor=0
#logquiver=0

#wanted = 'gamma'
#wanted = 'temp'
#wanted = 'density'
wanted = 'cosmic_ray'
#wanted = 'Bfield'

#wanted='ke'

#wanted='zvel'


#remark='MWLRMHDIC_x'


galname='CRdiff3D'
#galname='m12icr_700'
#galname='m12icr_b_70'
#galname='mw'
#galname='smc'
#galname='sbc'
remark=Nsnapstring
#remark='mrmhd'
#remark='mrthree'
#remark='mrshort'
#remark=str(Nsnap)
#remark='lrmhd'
#remark='streamlr'
#remark='mrz'

#remark = 'CRDC_hole_z'

#mode = 'Slice'
mode = 'projected'

needquiver=''
#needquiver='v'
#needquiver='B'

projectionaxis='y'

newlabelneed=1

labfs=10
labs=10

if needquiver=='B':
        logquiver=1
        headaxislength=0
        headwidth=0
        headlength=0
else:
        logquiver=0
        headaxislength=4.5
        headwidth=3.
        headlength=5.


fig = plt.figure()

unit_base = {'UnitLength_in_cm'         : 3.08568e+21,
             'UnitMass_in_g'            :   1.989e+43,
             'UnitVelocity_in_cm_per_s' :      100000}


bbox_lim = 6 #kpc

bbox = [[-bbox_lim,bbox_lim],
        [-bbox_lim,bbox_lim],
        [-bbox_lim,bbox_lim]]

#newboxsize=75
 
newboxsize=10

projectionaxis='z'

nrows=1
ncols=len(fns)
#if len(fns)>4:
#       nrows = 2
#       ncols = 4
#if len(fns)>8:
#        nrows = 3
#        ncols = 4
if len(fns)>3:
        nrows = 2
        ncols = 3
if len(fns)>6:
        nrows = 3
        ncols = 3

#grid = AxesGrid(fig, (0.07,0.07,0.8,0.8),
grid = AxesGrid(fig, 111,
                nrows_ncols = (nrows, ncols),
                axes_pad = 0.05,
                label_mode = "L",
                share_all = True,
                cbar_location="right",
                cbar_mode="single",
                cbar_size="3%",
                cbar_pad="0%")

def symlog(x):
    """ Returns the symmetric log10 value """
    return np.sign(x) * np.log10(np.abs(x))


for i, fname in enumerate(fns):
        print 'loading', fname
        ds = yt.load(fname,unit_base=unit_base,bounding_box=bbox)
        #ds = GizmoDataset(fname,unit_base=unit_base,bounding_box=bbox)
        ds.index
        ad= ds.all_data()
        sorted(ds.field_list)
        #print 'ds.field_list', ds.field_list
        ad = ds.all_data()


        center = ds.arr([0,0,0],'kpc')
        new_box_size = ds.quan(newboxsize,'kpc')
        left_edge = center - new_box_size/2
        right_edge = center + new_box_size/2
        ad2= ds.region(center=center, left_edge=left_edge, right_edge=right_edge)
        sp = ds.sphere(center, (6, 'kpc'))
        #angmom = sp.quantities.angular_momentum_vector(use_gas=True,use_particles=False)

                
        if (wanted == 'temp'):
                if mode == 'projected':
                        px = yt.ProjectionPlot(ds, projectionaxis, ('gas', 'temperature'), center=center, width=new_box_size, weight_field=('gas','density'))
                        cblabel = r'projected ${\rm T \;[K]}$'
                if mode == 'Slice':
                        px = yt.SlicePlot(ds, projectionaxis, ('gas', 'temperature'), center=center, width=new_box_size)
                        cblabel = r'${\rm T \;[K]}$'
                px_frb = px.data_source.to_frb((newboxsize, "kpc"), 128)
                px_dens = np.array(px_frb[ ('gas', 'temperature')])
                cbcolor = 'hot'

        if (wanted == 'Bfield'):
                def _Benergydensityfun(field, data):
                    Bf = data['PartType0', 'MagneticField'].in_cgs()
                    print 'Bf.shape', Bf.shape
                    Bdensity = (Bf[:,0]*Bf[:,0]+Bf[:,1]*Bf[:,1]+Bf[:,2]*Bf[:,2])/8./np.pi
                    return Bdensity
                ds.add_field(('PartType0', 'Benergydensity'), function=_Benergydensityfun,
                             units="erg/cm**3", particle_type=True, display_name="Benergydensity")

                add_volume_weighted_smoothed_field('PartType0', 'Coordinates', 'Masses',
                                                   'SmoothingLength', 'Density',
                                                   'Benergydensity', ds.field_info)
                ad2= ds.region(center=center, left_edge=left_edge, right_edge=right_edge)
                if mode == 'projected':
                        px = yt.ProjectionPlot(ds, projectionaxis, ('deposit', 'PartType0_smoothed_Benergydensity'), center=center, width=new_box_size, north_vector=north_vector)
                if mode == 'Slice':
                        px = yt.SlicePlot(ds, projectionaxis, ('deposit', 'PartType0_smoothed_Benergydensity'), center=center, width=new_box_size)
                px_frb = px.data_source.to_frb((newboxsize, "kpc"), 128)
                px_dens = np.array(px_frb[('deposit', 'PartType0_smoothed_Benergydensity')])
                cbcolor = 'RdPu'
                cblabel = r'${\rm B\;energy\;[ erg/cm^3}]$'


        if wanted=='density':
                if mode == 'projected':
                        px = yt.ProjectionPlot(ds, projectionaxis, ('gas', 'density'), center=center, width=new_box_size)
                if mode == 'Slice':
                        px = yt.SlicePlot(ds, projectionaxis, ('gas', 'density'), center=center, width=new_box_size)    
                px_frb = px.data_source.to_frb((newboxsize, "kpc"), 128)
                px_dens = np.array(px_frb[('gas', 'density')])
                cbcolor='YlOrRd'
                cblabel=r'${\rm \log (\rho [g/cm^3])}$'

        if (wanted == 'cosmic_ray'):
                def _cosmicrayenergydensity(field, data):
                    energy = data['PartType0', 'CosmicRayEnergy']
                    energy = yt.YTArray(1e10*Msun_in_g*km_in_cm*km_in_cm*erg_in_eV*energy,"eV")
                    mass = data['PartType0', 'Masses'].in_cgs()
                    density = data['PartType0', 'Density'].in_cgs()
                    crdensity = energy/mass*density
                    return crdensity
                ds.add_field(('PartType0', 'cosmicrayenergydensity'), function=_cosmicrayenergydensity,
                    units="eV/cm**3", particle_type=True)
                add_volume_weighted_smoothed_field('PartType0', 'Coordinates', 'Masses',
                                   'SmoothingLength', 'Density',
                                   'cosmicrayenergydensity', ds.field_info)
                px = yt.ParticlePlot(ds, 'particle_position_x', 'particle_position_y',
                    'particle_mass', weight_field='particle_ones')
                if mode=='projected':
                        px = yt.ProjectionPlot(ds, projectionaxis, ('deposit', 'PartType0_smoothed_cosmicrayenergydensity'), center=center, width=new_box_size)
                if mode=='Slice':
                        px = yt.SlicePlot(ds, projectionaxis, ('deposit', 'PartType0_smoothed_cosmicrayenergydensity'), center=center, width=new_box_size)
                #px.set_unit(('deposit', 'PartType0_smoothed_cosmicrayenergydensity'), 'eV/cm**3')
                px_frb = px.data_source.to_frb((newboxsize, "kpc"), 128)
                px_dens = np.array(px_frb[('deposit', 'PartType0_smoothed_cosmicrayenergydensity')])
                cbcolor = 'YlGnBu'
                cblabel = r'${\rm \log (e_{CR} [eV/cm^3])}$'


        if (wanted=='gamma'):
                def _gammaenergydensity(field, data):
                    #Fcal = 7e-2
                    energy = data['PartType0', 'CosmicRayEnergy']
                    energy = yt.YTArray(energy*2e53, "g*cm**2/s**2")
                    mass = data['PartType0', 'Masses'].in_cgs()
                    density = data['PartType0', 'Density'].in_cgs()
                    crdensity = energy/mass*density
                    tpi = 2e5/density*yt.YTArray(250.0,"g/cm**3")
                    gammadensity = crdensity*0.25/tpi/3.2e7*0.7/3.0
                    return gammadensity
                ds.add_field(('PartType0', 'gammaenergydensity'), function=_gammaenergydensity,
                             units="erg/cm**3", particle_type=True, display_name="Gamma ray Energy Density")
                add_volume_weighted_smoothed_field('PartType0', 'Coordinates', 'Masses',
                                                   'SmoothingLength', 'Density',
                                                   'gammaenergydensity', ds.field_info)
                ds.index
                ad= ds.all_data()
                center = ds.arr([0,0,0],'kpc')
                new_box_size = ds.quan(newboxsize,'kpc')
                left_edge = center - new_box_size/2
                right_edge = center + new_box_size/2
                ad2= ds.region(center=center, left_edge=left_edge, right_edge=right_edge)
                if mode=='projected':
                        px = yt.ProjectionPlot(ds, projectionaxis, ('deposit', 'PartType0_smoothed_gammaenergydensity'), center=center, width=new_box_size)
                #       px.set_zlim('all',1e-28,1e-38)
                        px.set_colorbar_label(('deposit', 'PartType0_smoothed_gammaenergydensity'),r'Projected $\rho_{\gamma}\; \mathrm{(\frac{erg}{cm^2\;s})}$')
                        cblabel = r'Projected $\rho_{\gamma}\; \mathrm{(\frac{erg}{cm^2\; s})}$'
                if mode=='Slice':
                        px = yt.SlicePlot(ds, projectionaxis, ('deposit', 'PartType0_smoothed_gammaenergydensity'), center=center, width=new_box_size)
                        px.set_colorbar_label(('deposit', 'PartType0_smoothed_gammaenergydensity'),r'$\rho_{\gamma}\; \mathrm{(\frac{erg}{cm^3})}$')
                        cblabel = r'$\rho_{\gamma}\; \mathrm{(\frac{erg}{cm^3})}$'
                px_frb = px.data_source.to_frb((newboxsize, "kpc"), 128)
                px_dens = np.array(px_frb[('deposit', 'PartType0_smoothed_gammaenergydensity')])
                cbcolor = 'hot'

        x_min, x_max, y_min, y_max = px_frb.bounds
        # Extract X, Y, U, V from the frb
        logpx_dens = np.nan_to_num(np.log10(px_dens))
        #px_dens = np.nan_to_num(px_dens)
        if i==0:
                vmax=np.amax(logpx_dens)
                vmin=np.amin(logpx_dens)
                print 'vmax', vmax
                print 'vmin', vmin
                if wanted=='temp':
                        if galname=='smc':
                                vmax=6.0
                                vim=1.0
                        else:
                                vmax=6.5
                                vmin=3.0
                if wanted=='density':
                        vmax=-25.0
                        vmin=vmax-4.0
                if wanted=='cosmic_ray':
                        vmax=21.5
                        vmin=vmax-2.0
                if wanted=='gamma':
                        if vmin-vmax<-20:
                                vmin=vmax-20
                if wanted=='Bfield':
                        if vmin-vmax<-20:
                                vmin=vmax-20
        #if wanted=='cosmic_ray':
        #    vmax=21.5
        #    vmin=vmax-7.0
        # Use matplotlib from there
        print 'vmax,vmin', vmax,vmin
        im = grid[i].imshow(logpx_dens, origin='lower', vmax=vmax, vmin=vmin, extent=[x_min,x_max,y_min,y_max], cmap=cbcolor)
        qsep=10
        if len(fns)<4:
                ineed=0
        else:
                ineed=3
        grid[i].set_xlabel('kpc',fontsize=labfs)
        grid[i].set_ylabel('kpc',fontsize=labfs)
        grid[i].tick_params(axis='x', labelsize=labs)
        grid[i].tick_params(axis='y', labelsize=labs)
        #grid[i].set_xticks(np.arange(-newboxsize/2.+10.,newboxsize/2.-10.,20.))

cbar = grid.cbar_axes[0].colorbar(im, extend='max')
cbar.ax.set_ylabel(cblabel, rotation=270,fontsize=12, labelpad=15)
cbar.ax.tick_params(labelsize=10)

#if quivercolor==1: 
#       if needquiver=='B':
#               cblabel=r'${\rm log(G)}$'
#       cbarq = grid.cbar_axes[0].colorbar(Q, pad=0.2)
#       cbar.ax.set_ylabel(cblabel, rotation=270,fontsize=12, labelpad=15)
#       cbar.ax.tick_params(labelsize=10)


#for cax in grid.cbar_axes:
#    cax.toggle_label(False)

print 'plotit, plotupt', plotit, plotupt
for ax, im_title, up_title in zip(grid, plotit, plotupt):
      t = add_inner_title(ax, "\n".join(wrap(im_title,17)), loc=3)
      t.patch.set_alpha(0.5)
      ax.set_title(up_title,fontsize=labfs)
                
if needquiver=='v':
        mode+='_vquiver'
if needquiver=='B':
        mode+='_Bquiver'
if headaxislength==0:
        mode+='_nohead'

#px.save('gas'+wanted+'_'+mode+'_'+remark+'.pdf')
YTdir = '/home/tkc004/scratch/testyt/'
directory = YTdir+'YTplot/'+galname
if not os.path.exists(directory):
    os.makedirs(directory)
directory = YTdir+'YTplot/'+galname+'/'+wanted
if not os.path.exists(directory):
    os.makedirs(directory)
filename=YTdir+'YTplot/'+galname+'/'+wanted+'/'+'gas'+wanted+'_'+mode+'_'+galname+remark+'.pdf'
print 'filename', filename
plt.savefig(filename,bbox_inches='tight',pad_inches = 0.1)
plt.clf()


yt : [INFO     ] 2019-03-02 16:14:36,685 Omega Lambda is 0.0, so we are turning off Cosmology.
yt : [INFO     ] 2019-03-02 16:14:36,723 Parameters: current_time              = 0.004999999888241291
yt : [INFO     ] 2019-03-02 16:14:36,725 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2019-03-02 16:14:36,726 Parameters: domain_left_edge          = [-6. -6. -6.]
yt : [INFO     ] 2019-03-02 16:14:36,728 Parameters: domain_right_edge         = [6. 6. 6.]
yt : [INFO     ] 2019-03-02 16:14:36,729 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2019-03-02 16:14:36,734 Allocating for 1.678e+07 particles (index particle type 'all')


loading /home/tkc004/oasis/bw/CRdifftest/dc28g3D_old/output/snapshot_005.hdf5


yt : [INFO     ] 2019-03-02 16:14:39,094 Identified 1.404e+06 octs
yt : [INFO     ] 2019-03-02 16:14:41,319 xlim = -6.000000 6.000000
yt : [INFO     ] 2019-03-02 16:14:41,320 ylim = -6.000000 6.000000
yt : [INFO     ] 2019-03-02 16:14:41,322 xlim = -6.000000 6.000000
yt : [INFO     ] 2019-03-02 16:14:41,324 ylim = -6.000000 6.000000
yt : [INFO     ] 2019-03-02 16:14:41,326 Splatting (('all', 'particle_mass')) onto a 800 by 800 mesh
yt : [WARNING  ] 2019-03-02 16:14:51,240 Plot image for field ('all', 'particle_mass') has zero dynamic range. Min = Max = 2904564198757950839673940845044695040.000000.
yt : [WARNING  ] 2019-03-02 16:14:51,241 Switching to linear colorbar scaling.
yt : [INFO     ] 2019-03-02 16:20:14,125 Projection completed
yt : [INFO     ] 2019-03-02 16:20:14,129 xlim = -5.000000 5.000000
yt : [INFO     ] 2019-03-02 16:20:14,130 ylim = -5.000000 5.000000
yt : [INFO     ] 2019-03-02 16:20:14,133 xlim = -5.000000 5.000000
yt : [INFO     ] 2019-03-02 16:20:14,134 ylim = -5.00

vmax 21.681259567212834
vmin -3.9831760415005784
vmax,vmin 21.5 19.5
loading /home/tkc004/oasis/bw/CRdifftest/M1dc28g3D_c1000_useflux_Jianhll_referee/output/snapshot_005.hdf5


yt : [INFO     ] 2019-03-02 16:20:14,754 Omega Lambda is 0.0, so we are turning off Cosmology.
yt : [INFO     ] 2019-03-02 16:20:14,790 Parameters: current_time              = 0.004999999888241291
yt : [INFO     ] 2019-03-02 16:20:14,791 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2019-03-02 16:20:14,793 Parameters: domain_left_edge          = [-6. -6. -6.]
yt : [INFO     ] 2019-03-02 16:20:14,794 Parameters: domain_right_edge         = [6. 6. 6.]
yt : [INFO     ] 2019-03-02 16:20:14,796 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2019-03-02 16:20:14,800 Allocating for 1.678e+07 particles (index particle type 'all')
yt : [INFO     ] 2019-03-02 16:20:17,171 Identified 1.404e+06 octs
yt : [INFO     ] 2019-03-02 16:20:19,449 xlim = -6.000000 6.000000
yt : [INFO     ] 2019-03-02 16:20:19,450 ylim = -6.000000 6.000000
yt : [INFO     ] 2019-03-02 16:20:19,452 xlim = -6.000000 6.000000
yt : [INFO     ] 2019-03-02 16:20:19,453 ylim = -6.000000 6.000000
yt : 

vmax,vmin 21.5 19.5
plotit, plotupt ['Zeroth moment', 'Two moment'] ['', '']
filename /home/tkc004/scratch/testyt/YTplot/CRdiff3D/cosmic_ray/gascosmic_ray_projected_CRdiff3D005.pdf


<Figure size 432x288 with 0 Axes>